## 基本方程

二维声波方程为

$$
\ddot{p}(x,z,t) \ = \ c(x,z)^2 (\partial_x^2 p(x,z,t) + \partial_z^2 p(x,z,t)) \ + s(x,z,t)
$$

我们用时间相关的部分（时间上标，空间下标）替换为

$$
 \frac{p_{j,k}^{n+1} - 2 p_{j,k}^n + p_{j,k}^{n-1}}{\mathrm{d}t^2} \ = \ c_j^2 ( \partial_x^2 p + \partial_z^2 p) \ + s_{j,k}^n
$$

解得 $p_{j,k}^{n+1}$。外推方案为

$$
p_{j,k}^{n+1} \ = \ c_j^2 \mathrm{d}t^2 \left[ \partial_x^2 p + \partial_z^2 p \right]
+ 2p_{j,k}^n - p_{j,k}^{n-1} + \mathrm{d}t^2 s_{j,k}^n
$$

空间导数由以下公式决定

$$
\partial_x^2 p \ = \ \frac{p_{j+1,k}^{n} - 2 p_{j,k}^n + p_{j-1,k}^{n}}{\mathrm{d}x^2}
$$
$$
\partial_z^2 p \ = \ \frac{p_{j,k+1}^{n} - 2 p_{j,k}^n + p_{j,k-1}^{n}}{\mathrm{d}z^2} 
$$

---"


### 1. 入门指南
将时间外推循环与我们在课程中开发的数值算法相关联。了解模拟的输入参数以及生成的图表。修改源和接收器位置，并观察地震图的影响。

### 2. 稳定性
Courant准则定义为 $eps = (v \cdot dt) / dx$，提供了最大可能的稳定时间步长，其中 $v$ 是速度，$dt$ 是时间步长，$dx$ 是空间步长。通过增加时间步长来尽可能精确地确定代码的稳定性限制。在每个时间步长打印压力场的最大值，并观察在稳定和不稳定模拟情况下的演变。

### 3. 高阶算子
通过添加使用5点差分算子的选项来扩展代码。比较使用3点和5点算子的模拟结果。稳定性限制是否仍然相同？估计每个波长的点数，并通过查找结果地震图中的数值色散迹象来调查模拟的准确性。5点权重为：$[-1/12, 4/3, -5/2, 4/3, -1/12]/dx^2$。

### 4. 数值各向异性
通过改变 $f_0$ 来增加波场的频率。调查波场的角度依赖性。波场为什么呈现各向异性？哪个方向最准确，为什么？如果将源时函数设置为尖峰（除了一个元素值为$1$的地方其他地方为零），会发生什么？

### 5. 异质模型
现在让我们通过改变模型的内部结构来探索有限差分法的能力。我们只能修改可以在每个网格点变化的速度 $c$（有限制吗？）。这里有一些建议：通过分析快照和地震图来研究结构对结果的影响。

* 在表面附近添加一个低（高）速层。将源放置在$z_s=2$处。
* 添加一个垂直的低速区域（断层带），宽度为一定宽度（例如$10$个网格点），并讨论所得波场
* 通过将表面以上的压力设置为$0$来模拟地形。使用高斯山丘形状或随机地形。
* 等等。

### 6. 源-接收器互易性
初始化一个强烈异质的二维速度模型，并模拟从内部源点（$x_s, z_s$）到内部接收器（$x_r, z_r$）传播的波。展示通过颠倒源和接收器，可以获得相同地震图的事实。

### 7. 时间反演
时间反演。在任意二维速度模型中，在域的中心定义一个源，并在源周围的适当距离处定义一个接收器圆。模拟一个波场，将其记录在接收器环上并存储结果。颠倒合成地震图，并将其作为接收器点的源注入。会发生什么？您知道使用此原理的例子吗？

---"


In [1]:
# This is a configuration step for the exercise. Please run it before the simulation code! 
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Simple finite difference solver 
# Acoustic wave equation  p_tt = c^2 p_xx + src
# 2-D regular grid

nx = 200      # grid points in x - 500
nz = 200      # grid points in z - 500
nt = 1000     # number of time steps
dx = 10.0     # grid increment in x - 1
dt = 0.001    # Time step
c0 = 3000.0   # velocity (can be an array) - 580
isx = nx // 2 # source index x - 250
isz = nz // 2 # source index z - 250
ist = 100     # shifting of source time function
f0 = 150.0     # dominant frequency of source (Hz)
isnap = 10    # snapshot frequency
T = 1.0 / f0  # dominant period
nop = 5       # length of operator

# Model type, available are "homogeneous", "fault_zone",
# "surface_low_velocity_zone", "random", "topography",
# "slab"
model_type = "fault_zone"

# Receiver locations
irx = np.array([60, 80, 100, 120, 140])
irz = np.array([5, 5, 5, 5, 5])
seis = np.zeros((len(irx), nt))

# Initialize pressure at different time steps and the second
# derivatives in each direction
p = np.zeros((nz, nx))
pold = np.zeros((nz, nx))
pnew = np.zeros((nz, nx))
pxx = np.zeros((nz, nx))
pzz = np.zeros((nz, nx))


In [3]:
# Initialize velocity model (the fun bit!)
c = np.zeros((nz, nx))

if model_type == "homogeneous":
    c += c0
elif model_type == "fault_zone":
    c += c0
    c[:, nx // 2 - 5: nx // 2 + 5] *= 0.8    
elif model_type == "surface_low_velocity_zone":
    c += c0
    c[1:10,:] *= 0.8
elif model_type == "random":
    pert = 0.4
    r = 2.0 * (np.random.rand(nz, nx) - 0.5) * pert
    c += c0 * (1 + r)   
elif model_type == "topography":
    c += c0
    c[0 : 10, 10 : 50] = 0                         
    c[0 : 10, 105 : 115] = 0                       
    c[0 : 30, 145 : 170] = 0
    c[10 : 40, 20 : 40]  = 0
    c[0 : 15, 50 : 105] *= 0.8    
elif model_type == "slab":
    c += c0
    c[110 : 125, 0 : 125] = 1.4 * c0
    for i in range(110, 180):
        c[i , i-5 : i + 15 ] = 1.4 * c0
else:
    raise NotImplementedError
    
cmax = c.max()

In [4]:
# Source time function Gaussian, nt + 1 as we loose the last one by diff
src = np.empty(nt + 1)
for it in range(nt):
    src[it] = np.exp(-1.0 / T ** 2 * ((it - ist) * dt) ** 2)
# Take the first derivative
src = np.diff(src) / dt
src[nt - 1] = 0

In [5]:
# Plot preparation

v = max([np.abs(src.min()), np.abs(src.max())])
# Initialize animated plot
image = plt.imshow(pnew, interpolation='nearest', animated=True,
                   vmin=-v, vmax=+v, cmap=plt.cm.RdBu)


# Plot the receivers
for x, z in zip(irx, irz):
    plt.text(x, z, '+')

plt.text(isx, isz, 'o')
plt.colorbar()
plt.xlabel('ix')
plt.ylabel('iz')


plt.ion()
#plt.show(block=False)

# required for seismograms
ir = np.arange(len(irx))

# Output Courant criterion
print("Courant Criterion eps :")
print(cmax*dt/dx)

<IPython.core.display.Javascript object>

Courant Criterion eps :
0.3


In [6]:
# Time extrapolation
for it in range(nt):
    if nop==3:
        # calculate partial derivatives, be careful around the boundaries
        for i in range(1, nx - 1):
            pzz[:, i] = p[:, i + 1] - 2 * p[:, i] + p[:, i - 1]
        for j in range(1, nz - 1):
            pxx[j, :] = p[j - 1, :] - 2 * p[j, :] + p[j + 1, :]

    if nop==5:
        # calculate partial derivatives, be careful around the boundaries
        for i in range(2, nx - 2):
            pzz[:, i] = -1./12*p[:,i+2]+4./3*p[:,i+1]-5./2*p[:,i]+4./3*p[:,i-1]-1./12*p[:,i-2]
        for j in range(2, nz - 2):
            pxx[j, :] = -1./12*p[j+2,:]+4./3*p[j+1,:]-5./2*p[j,:]+4./3*p[j-1,:]-1./12*p[j-2,:]
                    
            
    pxx /= dx ** 2
    pzz /= dx ** 2

    # Time extrapolation
    pnew = 2 * p - pold + dt ** 2 * c ** 2 * (pxx + pzz)
    # Add source term at isx, isz
    pnew[isz, isx] = pnew[isz, isx] + src[it]

    # Plot every isnap-th iteration
    if it % isnap == 0:    # you can change the speed of the plot by increasing the plotting interval
        
        plt.title("Max P: %.2f" % p.max())
        image.set_data(pnew)
        plt.gcf().canvas.draw()

    pold, p = p, pnew

    # Save seismograms
    seis[ir, it] = p[irz[ir], irx[ir]]

<IPython.core.display.Javascript object>

In [7]:
# Plot the source time function and the seismograms 

plt.ioff()
plt.figure(figsize=(8, 8))

plt.subplot(221)
time = np.arange(nt) * dt
plt.plot(time, src)
plt.title('Source time function')
plt.xlabel('Time (s) ')
plt.ylabel('Source amplitude ')

#plt.subplot(222)
#ymax = seis.ravel().max()  
#for ir in range(len(seis)):
#    plt.plot(time, seis[ir, :] + ymax * ir)
#    plt.xlabel('Time (s)')
#    plt.ylabel('Amplitude')

plt.subplot(223)
ymax = seis.ravel().max()
for ir in range(len(seis)):
    plt.plot(time, seis[ir, :] + ymax * ir)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

plt.subplot(224)
# The velocity model is influenced by the Earth model above
plt.title('Velocity Model')
plt.imshow(c)
plt.xlabel('ix')
plt.ylabel('iz')
plt.colorbar()

plt.show()

<IPython.core.display.Javascript object>